In [ ]:
from src.dl.loader import PatchRedressDataset, BlockRegressDataset, BlockClassifyDataset
import numpy as np
from src.dl.resnet import RegressResNet18, RegressResNet3d
from torch import nn
from torch.utils.data import DataLoader, sampler
from torch.utils.tensorboard import SummaryWriter
import torch
from sklearn.model_selection import KFold
import copy

In [ ]:
writer = SummaryWriter('./log')

In [ ]:
net = RegressResNet3d().cuda()

In [ ]:
import os
import os.path
net = torch.load(os.path.join('data', 'bin', 'resnet50.pth'))

In [ ]:
loss_fn = nn.SmoothL1Loss().cuda()
lr = 1e-3
optim = torch.optim.SGD([
    {'params': net.fc.weight, 'lr': 1e-2}
    ], lr=lr, momentum=0.3)
#optim = torch.optim.SGD(net.parameters(), lr=1e-1, momentum=0.3)
#scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, gamma=0.99)
#optim = torch.optim.Adam(net.parameters(), lr=lr, betas=[0.3, 0.1])
epoch = 100

In [ ]:
dataset = BlockRegressDataset('train')
kf = KFold(n_splits=5, shuffle=True, random_state=10)

In [ ]:
for fold, (train_idx, val_idx) in enumerate(kf.split(np.arange(len(dataset)))):
    train_sampler = sampler.SubsetRandomSampler(train_idx)
    val_sampler = sampler.SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=64, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=64, sampler=val_sampler)
    for i in range(epoch):
        total_loss = 0
        predy = np.array([])
        y = np.array([])
        net.train()
        for step, [img, labels, score] in enumerate(train_loader):
            img = img.cuda()
            labels = labels.cuda()
            score = score.cuda()
            output = net(img, labels).squeeze(-1)
            loss = loss_fn(output.float(), score.float())

            optim.zero_grad()
            loss.backward()
            optim.step()
            #scheduler.step()
        
        net.eval()
        with torch.no_grad():
            for step, [img, labels, score] in enumerate(val_loader):
                img = img.cuda()
                labels = labels.cuda()
                score = score.cuda()
                output = net(img, labels).squeeze(-1)
                loss = loss_fn(output.float(), score.float())
        
                predy = np.concatenate((predy, output.cpu().detach().numpy()), axis=0)
                y = np.concatenate((y, score.cpu().detach().numpy()), axis=0)

                total_loss += loss.item()

        nmse = np.sum((y - predy) ** 2) / np.sum(y ** 2)
        mse = np.sum((y - predy) ** 2) / len(train_idx)
        mae = np.sum(np.abs(y - predy)) / len(train_idx)
        meany = np.mean(y)
        rsquared = np.sum((predy - meany) ** 2) / np.sum((y - meany) ** 2)
        print('epoch {}, loss {}, nmse {}, mae {}, rsquared {}'.format(i, total_loss, nmse, mae, rsquared))
        writer.add_scalar('Loss', total_loss, i)
        writer.add_scalar('NMSE', nmse, i)
        writer.add_scalar('MAE', mae, i)
        writer.add_scalar('rsquared', rsquared, i)

writer.close()

In [ ]:
predy = []
y = []
for fold, (train_idx, val_idx) in enumerate(kf.split(np.arange(len(dataset)))):
    train_sampler = sampler.SubsetRandomSampler(train_idx)
    val_sampler = sampler.SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=64, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=64, sampler=val_sampler)
    net.eval()
    with torch.no_grad():
        for step, [img, labels, score] in enumerate(train_loader):
            img = img.cuda()
            labels = labels.cuda()
            score = score.cuda()
            output = net(img, labels).squeeze(-1)
            loss = loss_fn(output.float(), score.float())
    
            predy = np.concatenate((predy, output.cpu().detach().numpy()), axis=0)
            y = np.concatenate((y, score.cpu().detach().numpy()), axis=0)

            total_loss += loss.item()

    nmse = np.sum((y - predy) ** 2) / np.sum(y ** 2)
    mse = np.sum((y - predy) ** 2) / len(train_idx)
    mae = np.sum(np.abs(y - predy)) / len(train_idx)
    meany = np.mean(y)
    rsquared = np.sum((predy - meany) ** 2) / np.sum((y - meany) ** 2)
    print(rsquared)
    print(mse)
    print(mae)
    break